In [1]:
import argparse
import logging
import matplotlib
import copy
import os
from matplotlib import pyplot as plot
from s4d.utils import *
from s4d.diar import Diar
from s4d import viterbi, segmentation
from s4d.clustering import hac_bic
from sidekit.sidekit_io import init_logging
from s4d.gui.dendrogram import plot_dendrogram
from s4d.diar import Diar
from s4d.utils import *
from s4d import scoring
from s4d.model_iv import ModelIV
from sidekit.sidekit_io import *
from sidekit.bosaris import IdMap, Scores
from s4d.clustering.hac_iv import hac_iv
from s4d import scoring
import numpy as np
import traceback

/home/usman/sde/venv3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Import theano


ERROR (theano.gpuarray): pygpu was configured but could not be imported or is too old (version 0.7 or higher required)
NoneType: None


In [2]:
loglevel = logging.DEBUG
init_logging(level=loglevel)

## Setting important directories

In [3]:
show = 'ES2003a.Mix-Headset'
dir = 'out-mfcc-with-vad-vit'
#dir = 'out-mfcc-with-vad'
#dir = 'out'
wdir = os.path.join(dir, show)
sdir = os.path.join(wdir, 'segments')
pdir = os.path.join(wdir, 'plda')
final_segments_dir = os.path.join(wdir, 'results')
linear_bic_dir = os.path.join(wdir, 'linear.bic')
bic_hac_dir = os.path.join(wdir, 'bic.hac')
hac_iv_dir = os.path.join(wdir, 'hac.iv')
results_vit_dir = os.path.join(wdir, 'results.vit')

if not os.path.exists(wdir):
    os.makedirs(wdir)
if not os.path.exists(sdir):
    os.makedirs(sdir)    
if not os.path.exists(pdir):
    os.makedirs(pdir)
if not os.path.exists(final_segments_dir):
    os.makedirs(final_segments_dir)
if not os.path.exists(linear_bic_dir):
    os.makedirs(linear_bic_dir)    
if not os.path.exists(bic_hac_dir):
    os.makedirs(bic_hac_dir)
if not os.path.exists(hac_iv_dir):
    os.makedirs(hac_iv_dir)
if not os.path.exists(results_vit_dir):
    os.makedirs(results_vit_dir)    
    
input_show = os.path.join('audio', show + '.wav')    
segments_dir = 'segments'    
plda_dir = 'plda'    
data_dir = 'data'
mfcc_dir = 'mfcc'
idmap_fn = show + '.idmap.h5'
model_fn = 'data/model/ester_model_1024_300_150.h5'
hac_diar_fn = os.path.join(wdir, segments_dir, show + '.hac.{:.2f}.seg')
score_fn = os.path.join(wdir, plda_dir, show + '.score.plda.h5') 
mfcc_fn = os.path.join(wdir, mfcc_dir, show + '.mfcc.h5')

#Segmentation file's directories
init_seg = os.path.join(wdir, segments_dir, 'init.seg')
gd_seg = os.path.join(wdir, segments_dir, 'gd_seg_250.seg')
linear_bic_seg = os.path.join(linear_bic_dir, 'li_bic.{:.2f}.seg')
mfcc_m_speaker = os.path.join(wdir, mfcc_dir, show + '.{:.2f}.{:.2f}.test.mfcc.h5')
bic_hac_seg = os.path.join(bic_hac_dir, 'bic_hac.{:.2f}.{:.2f}.seg')
hac_iv_seg = os.path.join(final_segments_dir, 'bic_hac.{:.2f}.{:.2f}.{:.2f}.seg')
results_vit_seg = os.path.join(results_vit_dir, 'bic_hac.{:.2f}.{:.2f}.{:.2f}.vit.-250.seg')


## Loading VAD and cepstral features with delta and double delta. Donot run if precomputed!

In [4]:
fe = get_feature_extractor(input_show, type_feature_extractor='basic')
fe.save(show, output_feature_filename=mfcc_fn)


2018-09-24 00:03:13,198 - INFO - process part : 0.000000 1139.765375 1139.765375
2018-09-24 00:03:14,916 - INFO - no vad
2018-09-24 00:03:14,921 - INFO - !! size of signal cep: 0.000050 len 13 type size 4
2018-09-24 00:03:14,970 - INFO - [ True  True  True ...  True  True  True]
2018-09-24 00:03:15,134 - INFO - out-mfcc-with-vad-vit/ES2003a.Mix-Headset/mfcc/ES2003a.Mix-Headset.mfcc.h5


In [5]:
fs = get_feature_server(mfcc_fn, feature_server_type='sid')
cep, _ = fs.load(show)
cep[np.logical_not(_)] = 0

2018-09-24 00:03:15,142 - INFO - out-mfcc-with-vad-vit/ES2003a.Mix-Headset/mfcc ## ES2003a ## .Mix-Headset.mfcc.h5
2018-09-24 00:03:15,142 - INFO - feature extractor --> None
2018-09-24 00:03:15,143 - INFO - --------------------
2018-09-24 00:03:15,144 - INFO - None
2018-09-24 00:03:15,144 - INFO - --------------------
2018-09-24 00:03:15,145 - INFO - 	 show: empty 

	 input_feature_filename: empty 

	 feature_filename_structure: out-mfcc-with-vad-vit/ES2003a.Mix-Headset/mfcc/ES2003a.Mix-Headset.mfcc.h5 
	  
	  

	 Post processing options: 
		 mask: None  
		 feat_norm: cmvn_sliding 
		 dct_pca: False, dct_pca_config: (12, 12, None) 
		 sdc: False, sdc_config: (1, 3, 7) 
		 delta: True, double_delta: True, delta_filter: [ 0.25  0.5   0.25  0.   -0.25 -0.5  -0.25] 
		 rasta: False 
		 keep_all_features: True 

2018-09-24 00:03:15,146 - INFO - we are inside the dataset list
2018-09-24 00:03:15,147 - INFO - ('energy', 'cep')
2018-09-24 00:03:15,196 - INFO - We are logging your feat
2018-0

In [6]:
#Parameters
win_size = 250

#linear bic segmentation parameters
#postive value of delta bic indicates we need two separate models for the adjacent segments. 
#Here we define the threshodls
li_bic_p_start = .2
li_bic_p_stop = 2.0
li_bic_p_num = 10

#bic hac threshold for merging clusters
bic_hac_start = .5
bic_hac_end = 3
bic_hac_num = 10

#hac_iv threshold
t_min = -50
t_max = 100
t_num = 15

vit_penalty = -250


###  Perform 3 layers of segmentation. Initial ( complete file is segmented), Gaussian divergence based segmentaion, followed by linear bic segmentation. ivectors are extracted and then clustered in the below code

In [7]:
init_diar = segmentation.init_seg(cep, show)
Diar.write_seg(init_seg, init_diar)
gd_diar = segmentation.segmentation(cep, init_diar, win_size)
Diar.write_seg(gd_seg, gd_diar)

In [ ]:
#ToDo Add bic clustering followed by add more threshold for i-vector scoring. Store files in multiple formats
#ToDo Make code multi-threaded and scalable for GPU

In [ ]:
model_iv = ModelIV(model_fn)
f = open('error.log', 'a+')
counter = 0
for t1 in np.linspace(li_bic_p_start, li_bic_p_stop, li_bic_p_num):
    try:
        # Loading Segmentations with varying thresholds
        bicl_diar = segmentation.bic_linear(cep, gd_diar, t1, sr=False)
        Diar.write_seg(linear_bic_seg.format(t1), bicl_diar)
        
        # Bic HAC
        for bic_value in np.linspace(bic_hac_start, bic_hac_end, bic_hac_num):
            bic = hac_bic.HAC_BIC(cep, bicl_diar, bic_value, sr=False)
            bic_hac_diar = bic.perform(to_the_end=True)
            Diar.write_seg(bic_hac_seg.format(t1, bic_value), bic_hac_diar)
            vit_diar = viterbi.viterbi_decoding(cep, bic_hac_diar, vit_penalty)

            # Extracting features per speaker
            fe = get_feature_extractor(input_show, type_feature_extractor='sid')
            idmap_bic = fe.save_multispeakers(bic_hac_diar.id_map(), \
                                              output_feature_filename=mfcc_m_speaker.format(t1, bic_value) \
                                              , keep_all=False)

            # training i-vectors
            fs = get_feature_server(mfcc_m_speaker.format(t1, bic_value), 'sid')
            model_iv.train(fs, idmap_bic)
            # Using plda to gather scores
            distance = model_iv.score_plda_slow()
            distance.write(score_fn)
            scores = Scores(scores_file_name=score_fn)
            #cep_vit, _ = fs.load(input_show, input_feature_filename=mfcc_m_speaker.format(t1, bic_value)) 

            # Using AHC on calculated scores
            for hac_value in np.linspace(t_min, t_max, t_num):
                diar_iv, _, _ = hac_iv(bic_hac_diar, scores, threshold=hac_value)
                Diar.write_seg(hac_iv_seg.format(t1, bic_value, hac_value), diar_iv)
                
                #viterbi resegmentation
                vit_diar = viterbi.viterbi_decoding(cep, diar_iv, vit_penalty)
                Diar.write_seg(results_vit_seg.format(t1, bic_value, hac_value), vit_diar)
                
                
    except Exception as e:
        traceback.print_exc()
        print('hello')
        print("\n")
        counter += 1
        f.write(str(e))
        continue
f.close()    

2018-09-24 00:03:19,886 - DEBUG - linear next S0 S1: 1/177 1090.118566
2018-09-24 00:03:19,892 - DEBUG - linear next S1 S2: 2/177 1741.319786
2018-09-24 00:03:19,895 - DEBUG - linear next S2 S3: 3/177 1248.900823
2018-09-24 00:03:19,898 - DEBUG - linear next S3 S4: 4/177 1434.347436
2018-09-24 00:03:19,909 - DEBUG - linear next S4 S5: 5/177 1192.972282
2018-09-24 00:03:19,914 - DEBUG - linear next S5 S6: 6/177 1854.187282
2018-09-24 00:03:19,917 - DEBUG - linear next S6 S7: 7/177 1962.221612
2018-09-24 00:03:19,920 - DEBUG - linear next S7 S8: 8/177 1638.824244
2018-09-24 00:03:19,923 - DEBUG - linear next S8 S9: 9/177 1267.734381
2018-09-24 00:03:19,926 - DEBUG - linear next S9 S10: 10/177 1482.347815
2018-09-24 00:03:19,934 - DEBUG - linear next S10 S11: 11/177 1518.707561
2018-09-24 00:03:19,937 - DEBUG - linear next S11 S12: 12/177 1016.436550
2018-09-24 00:03:19,940 - DEBUG - linear next S12 S13: 13/177 977.996855
2018-09-24 00:03:19,944 - DEBUG - linear next S13 S14: 14/177 857.7

load:  data/model/ester_model_1024_300_150.h5


2018-09-24 00:03:20,083 - DEBUG - linear next S38 S39: 39/177 2150.735905
2018-09-24 00:03:20,087 - DEBUG - linear next S39 S40: 40/177 1264.358305
2018-09-24 00:03:20,090 - DEBUG - linear next S40 S41: 41/177 1147.565484
2018-09-24 00:03:20,114 - DEBUG - linear next S41 S42: 42/177 755.560087
2018-09-24 00:03:20,132 - DEBUG - linear next S42 S43: 43/177 939.050265
2018-09-24 00:03:20,139 - DEBUG - linear next S43 S44: 44/177 1358.297301
2018-09-24 00:03:20,143 - DEBUG - linear next S44 S45: 45/177 1143.361332
2018-09-24 00:03:20,146 - DEBUG - linear next S45 S46: 46/177 1098.741037
2018-09-24 00:03:20,163 - DEBUG - linear next S46 S47: 47/177 1019.970822
2018-09-24 00:03:20,168 - DEBUG - linear next S47 S48: 48/177 1373.629048
2018-09-24 00:03:20,171 - DEBUG - linear next S48 S49: 49/177 438.370191
2018-09-24 00:03:20,173 - DEBUG - linear next S49 S50: 50/177 598.426618
2018-09-24 00:03:20,176 - DEBUG - linear next S50 S51: 51/177 768.222374
2018-09-24 00:03:20,179 - DEBUG - linear ne

2018-09-24 00:03:20,907 - DEBUG - linear next S148 S149: 149/177 1654.698138
2018-09-24 00:03:20,908 - DEBUG - linear next S149 S150: 150/177 1175.920882
2018-09-24 00:03:20,910 - DEBUG - linear next S150 S151: 151/177 1254.882150
2018-09-24 00:03:20,912 - DEBUG - linear next S151 S152: 152/177 1831.479897
2018-09-24 00:03:20,932 - DEBUG - linear next S152 S153: 153/177 1654.178264
2018-09-24 00:03:20,936 - DEBUG - linear next S153 S154: 154/177 1168.787292
2018-09-24 00:03:20,939 - DEBUG - linear next S154 S155: 155/177 1635.438019
2018-09-24 00:03:20,942 - DEBUG - linear next S155 S156: 156/177 2507.513303
2018-09-24 00:03:20,945 - DEBUG - linear next S156 S157: 157/177 975.898093
2018-09-24 00:03:20,967 - DEBUG - linear next S157 S158: 158/177 1319.230334
2018-09-24 00:03:20,971 - DEBUG - linear next S158 S159: 159/177 1477.209864
2018-09-24 00:03:20,974 - DEBUG - linear next S159 S160: 160/177 1133.123670
2018-09-24 00:03:20,977 - DEBUG - linear next S160 S161: 161/177 1347.131217


2018-09-24 00:03:27,001 - DEBUG - merge: 66 c1: S143 (42) c2: S34 (79) dist: -236.363103 112
2018-09-24 00:03:27,037 - DEBUG - merge: 67 c1: S112 (16) c2: S13 (29) dist: -224.547939 111
2018-09-24 00:03:27,059 - DEBUG - merge: 68 c1: S102 (5) c2: S112 (16) dist: -258.444022 110
2018-09-24 00:03:27,088 - DEBUG - merge: 69 c1: S102 (5) c2: S21 (71) dist: -295.628883 109
2018-09-24 00:03:27,121 - DEBUG - merge: 70 c1: S102 (5) c2: S12 (20) dist: -350.757038 108
2018-09-24 00:03:27,151 - DEBUG - merge: 71 c1: S102 (5) c2: S167 (61) dist: -246.882094 107
2018-09-24 00:03:27,181 - DEBUG - merge: 72 c1: S102 (5) c2: S125 (23) dist: -257.739968 106
2018-09-24 00:03:27,222 - DEBUG - merge: 73 c1: S102 (5) c2: S147 (42) dist: -298.820921 105
2018-09-24 00:03:27,253 - DEBUG - merge: 74 c1: S102 (5) c2: S62 (91) dist: -240.009967 104
2018-09-24 00:03:27,271 - DEBUG - merge: 75 c1: S102 (5) c2: S157 (52) dist: -254.780426 103
2018-09-24 00:03:27,308 - DEBUG - merge: 76 c1: S102 (5) c2: S38 (74) dis

...

2018-09-24 00:52:14,002 - DEBUG - Maximisation
2018-09-24 00:52:14,005 - DEBUG - Expectation
2018-09-24 00:52:14,006 - DEBUG - No Parallel processing with this module
2018-09-24 00:52:14,121 - DEBUG - Maximisation
2018-09-24 00:52:14,124 - DEBUG - Expectation
2018-09-24 00:52:14,125 - DEBUG - No Parallel processing with this module
2018-09-24 00:52:14,238 - DEBUG - Maximisation
2018-09-24 00:52:14,241 - DEBUG - Expectation
2018-09-24 00:52:14,242 - DEBUG - No Parallel processing with this module
2018-09-24 00:52:14,356 - DEBUG - Maximisation
2018-09-24 00:52:14,359 - DEBUG - Expectation
2018-09-24 00:52:14,361 - DEBUG - No Parallel processing with this module
2018-09-24 00:52:14,475 - DEBUG - Maximisation
2018-09-24 00:52:14,720 - DEBUG - perform from 0 to 113974
2018-09-24 00:52:16,604 - DEBUG - c0: 0 c1: 1 value: 1.0000
2018-09-24 00:52:16,604 - DEBUG - 	 c0: S0 c1: S103 value: 1.0000
2018-09-24 00:52:16,679 - DEBUG - Expectation
2018-09-24 00:52:16,680 - DEBUG - No Parallel processi

2018-09-24 00:52:30,011 - DEBUG - Expectation
2018-09-24 00:52:30,013 - DEBUG - No Parallel processing with this module
2018-09-24 00:52:30,219 - DEBUG - Maximisation
2018-09-24 00:52:30,221 - DEBUG - Expectation
2018-09-24 00:52:30,222 - DEBUG - No Parallel processing with this module
2018-09-24 00:52:30,435 - DEBUG - Maximisation
2018-09-24 00:52:30,438 - DEBUG - Expectation
2018-09-24 00:52:30,440 - DEBUG - No Parallel processing with this module
2018-09-24 00:52:30,662 - DEBUG - Maximisation
2018-09-24 00:52:30,664 - DEBUG - Expectation
2018-09-24 00:52:30,665 - DEBUG - No Parallel processing with this module
2018-09-24 00:52:30,884 - DEBUG - Maximisation
2018-09-24 00:52:30,886 - DEBUG - Expectation
2018-09-24 00:52:30,889 - DEBUG - No Parallel processing with this module
2018-09-24 00:52:31,109 - DEBUG - Maximisation
2018-09-24 00:52:31,112 - DEBUG - Expectation
2018-09-24 00:52:31,114 - DEBUG - No Parallel processing with this module
2018-09-24 00:52:31,337 - DEBUG - Maximisatio

2018-09-24 00:52:43,917 - DEBUG - Maximisation
2018-09-24 00:52:43,920 - DEBUG - Expectation
2018-09-24 00:52:43,921 - DEBUG - No Parallel processing with this module
2018-09-24 00:52:44,021 - DEBUG - Maximisation
2018-09-24 00:52:44,025 - DEBUG - Expectation
2018-09-24 00:52:44,026 - DEBUG - No Parallel processing with this module
2018-09-24 00:52:44,154 - DEBUG - Maximisation
2018-09-24 00:52:44,158 - DEBUG - Expectation
2018-09-24 00:52:44,159 - DEBUG - No Parallel processing with this module
2018-09-24 00:52:44,269 - DEBUG - Maximisation
2018-09-24 00:52:44,272 - DEBUG - Expectation
2018-09-24 00:52:44,274 - DEBUG - No Parallel processing with this module
2018-09-24 00:52:44,386 - DEBUG - Maximisation
2018-09-24 00:52:44,389 - DEBUG - Expectation
2018-09-24 00:52:44,390 - DEBUG - No Parallel processing with this module
2018-09-24 00:52:44,503 - DEBUG - Maximisation
2018-09-24 00:52:44,506 - DEBUG - Expectation
2018-09-24 00:52:44,507 - DEBUG - No Parallel processing with this modul

2018-09-24 00:53:04,982 - INFO - ES2003a.Mix-Headset/S103 start: 0 stop: 47058
2018-09-24 00:53:09,915 - DEBUG - c0: 0 c1: 1 value: 1.0000
2018-09-24 00:53:09,917 - DEBUG - 	 c0: S0 c1: S103 value: 1.0000
2018-09-24 00:53:10,045 - DEBUG - Expectation
2018-09-24 00:53:10,047 - DEBUG - No Parallel processing with this module
2018-09-24 00:53:10,525 - DEBUG - Maximisation
2018-09-24 00:53:10,528 - DEBUG - Expectation
2018-09-24 00:53:10,529 - DEBUG - No Parallel processing with this module
2018-09-24 00:53:10,946 - DEBUG - Maximisation
2018-09-24 00:53:10,948 - DEBUG - Expectation
2018-09-24 00:53:10,949 - DEBUG - No Parallel processing with this module
2018-09-24 00:53:11,354 - DEBUG - Maximisation
2018-09-24 00:53:11,357 - DEBUG - Expectation
2018-09-24 00:53:11,358 - DEBUG - No Parallel processing with this module
2018-09-24 00:53:11,795 - DEBUG - Maximisation
2018-09-24 00:53:11,798 - DEBUG - Expectation
2018-09-24 00:53:11,800 - DEBUG - No Parallel processing with this module
2018-09

2018-09-24 00:53:38,335 - DEBUG - c0: 0 c1: 1 value: 1.0000
2018-09-24 00:53:38,335 - DEBUG - 	 c0: S0 c1: S103 value: 1.0000
2018-09-24 00:53:38,426 - DEBUG - Expectation
2018-09-24 00:53:38,426 - DEBUG - No Parallel processing with this module
2018-09-24 00:53:38,783 - DEBUG - Maximisation
2018-09-24 00:53:38,785 - DEBUG - Expectation
2018-09-24 00:53:38,786 - DEBUG - No Parallel processing with this module
2018-09-24 00:53:39,166 - DEBUG - Maximisation
2018-09-24 00:53:39,169 - DEBUG - Expectation
2018-09-24 00:53:39,169 - DEBUG - No Parallel processing with this module
2018-09-24 00:53:39,602 - DEBUG - Maximisation
2018-09-24 00:53:39,604 - DEBUG - Expectation
2018-09-24 00:53:39,606 - DEBUG - No Parallel processing with this module
2018-09-24 00:53:40,018 - DEBUG - Maximisation
2018-09-24 00:53:40,020 - DEBUG - Expectation
2018-09-24 00:53:40,021 - DEBUG - No Parallel processing with this module
2018-09-24 00:53:40,437 - DEBUG - Maximisation
2018-09-24 00:53:40,441 - DEBUG - Expec

In [ ]:
counter

### Compute DER

In [ ]:
import pandas as pd
from pyannote.core import PYANNOTE_URI, PYANNOTE_SEGMENT, PYANNOTE_LABEL, PYANNOTE_TRACK
from pyannote.core import Annotation, Segment

In [ ]:
def mdtm_convert_to_annote(filename=None, flag='seg'):
    cols = [PYANNOTE_TRACK, 'channel_no', PYANNOTE_SEGMENT,\
            'duration', 'speaker', 'trash1', 'trash2', PYANNOTE_LABEL]
    #df = pd.read_csv(filename, header=None, sep=' ', names=cols)
    if flag == 'seg':
        df = Diar.read_seg(filename)
    if flag == 'mdtm':
        df = Diar.read_mdtm(filename)
    temp_annotation = Annotation()
    rate = .1
    for row in df:
        start = row['start']*rate
        stop = row['stop']*rate
        label = row['cluster']
        filename = row['show']
        #print(start, stop, label, filename)
        temp_annotation[Segment(start, stop), filename] = label
    return temp_annotation.copy()    

In [ ]:
from pyannote.metrics.diarization import DiarizationErrorRate
result_dir = os.path.join(wdir, 'results')
files = os.listdir(result_dir)
ref = mdtm_convert_to_annote('es2003a', 'mdtm')
results = []
for file_name in files:
    result_file = os.path.join(result_dir, file_name)
    hyp = mdtm_convert_to_annote(result_file, 'seg')
    diarizationErrorRate = DiarizationErrorRate(skip_overlap=True, collar=.25)
    der = diarizationErrorRate(ref, hyp, uem=Segment(0, 1139))
    #print("DER = {0:.3f}".format(der))
    results.append(der)

In [ ]:
# Results for feature type basic with feature type SID. 4000 experiments
min(results)

In [ ]:
# Results for feature type when SID. 1640 experiments
min(results)

In [ ]:
# Perfect DER
diarizationErrorRate = DiarizationErrorRate()
der = diarizationErrorRate(ref, ref, uem=Segment(0, 1139))
print("DER = {0:.3f}".format(der))
